In [1]:
#Using this notebook requires set shell paths for SPARK_HOME, PYTHONPATH,
#SPARK_CLASSPATH as described in section Buildin Hail from Source at
#https://hail.is/docs/stable/getting_started.html

#The compiled Hail JAR file for logistic regression  requires HAIL
#implementation of Logistic mixed model regression as implemented in
#Hail fork ttasa-GLMM-1 at https://github.com/ttasa/hail/tree/ttasa-GLMM-1
#Jar file can be obtained by cloning the repository and 
#building the JAR running "./gradlew shadowJar" with shell.
from hail import *
hc = HailContext()

In [2]:
vds=hc.read("src/test/resources/LogitMM.vds").annotate_genotypes_expr("g = g.GT.toGenotype()")
vds.summarize()

Summary(samples=200, variants=200, call_rate=0.651550, contigs=['chr1'], multiallelics=0, snps=200, mnps=0, insertions=0, deletions=0, complex=0, star=0, max_alleles=2)

In [3]:
#Prepare variant dataset for analysis and test with running regular Hail linear regression.
annotations = hc.import_table("src/test/resources/LogitMM.annot1", types = {"pheno1": TFloat64(), "cov2" : TFloat64(),"cov3" : TFloat64()}).key_by("ID_1")
vds_result = vds.annotate_samples_table(annotations, root='sa.phenotypes')
vds_result = vds_result.linreg(['sa.phenotypes.pheno1'], covariates=['sa.phenotypes.cov2', 'sa.phenotypes.cov3'])
vds_result.summarize()

Summary(samples=200, variants=200, call_rate=0.651550, contigs=['chr1'], multiallelics=0, snps=200, mnps=0, insertions=0, deletions=0, complex=0, star=0, max_alleles=2)

In [4]:
 #Perform logistic mixed model regression ( Mixed model Polya-Gamma approach with expectation maximisation )
vds_result = vds_result.logmmreg('sa.phenotypes.pheno1', cov=['sa.phenotypes.cov2', 'sa.phenotypes.cov3'])

In [10]:
#Prints annotated variant dataset result
v_table=vds_result.globals
print (v_table)

Struct{u'logmmreg': Struct{u'c': 1.0, u'beta': {u'intercept': 0.5061125835792826, u'sa.phenotypes.cov3': -0.4217788267192358, u'sa.phenotypes.cov2': 0.20438210979155244}, u'phi': 0.007}}


In [11]:
#Perform  logistic mixedmodel analysis with different covariance parameters and 
#due to small size of data, use direct instead of approximate 
#linear equation solver (method: Direct).
vds_result = vds_result.logmmreg('sa.phenotypes.pheno1',cov=['sa.phenotypes.cov2', 'sa.phenotypes.cov3'],phi=2.0,c=2.3,optMethod="Direct")

In [7]:
#Print out new set of results
v_table=vds_result.globals
print (v_table)

Struct{u'logmmreg': Struct{u'c': 2.3, u'beta': {u'intercept': 0.5350008508464963, u'sa.phenotypes.cov3': -0.42513039478540937, u'sa.phenotypes.cov2': 0.18737730635415528}, u'phi': 2.0}}


In [8]:
##Generate data from Balding Nichols model
from hail.stats import TruncatedBetaDist
vds_2 = hc.balding_nichols_model(4, 200, 200, 2,
                             pop_dist=[0.1, 0.2, 0.3, 0.4],
                                fst=[.02, .06, .04, .12],
                                af_dist=TruncatedBetaDist(a=1.6, b=0.9, minVal=0.1, maxVal=0.95),seed=1)

In [21]:
#Annotate with previously used annotations
annotations = hc.import_table("src/test/resources/LogitMM.annot", types = {"pheno1": TFloat64(), "cov2" : TFloat64(),"cov3" : TFloat64()}).key_by("ID_1")

In [22]:
vds_2 = vds_2.annotate_samples_table(annotations, root='sa.phenotypes')

In [23]:
vds_result2 = vds_2.logmmreg('sa.phenotypes.pheno1', cov=['sa.phenotypes.cov2', 
     'sa.phenotypes.cov3'],phi=0.007,optMethod='LBFGS')

In [24]:
vds_result2 = vds_2.logmmreg('sa.phenotypes.pheno1', cov=['sa.phenotypes.cov2', 'sa.phenotypes.cov3'],
            phi=0.2,optMethod="Direct")

In [25]:
v_table=vds_result2.globals
print (v_table)

Struct{u'ancestralAFDist': Struct{u'a': 1.6, u'maxVal': 0.95, u'b': 0.9, u'type': u'TruncatedBetaDist', u'minVal': 0.1}, u'logmmreg': Struct{u'c': 1.0, u'beta': {u'intercept': 0.5063141361639452, u'sa.phenotypes.cov3': -0.4206565557494321, u'sa.phenotypes.cov2': 0.21050647010858073}, u'phi': 0.2}, u'popDist': [0.1, 0.2, 0.3, 0.4], u'nPops': 4, u'nVariants': 200, u'seed': 1, u'Fst': [0.02, 0.06, 0.04, 0.12], u'nSamples': 200}


In [32]:
#Export genotype data, samples and variants and print global annotations
vds_2.export_genotypes('output/genotypes.tsv', 's,v,g.gt ,sa.phenotypes.cov,sa.phenotypes.cov2,sa.phenotypes.cov3,sa.phenotypes.pheno1')
v_table=vds_result2.globals
print (v_table)

Struct{u'ancestralAFDist': Struct{u'a': 1.6, u'maxVal': 0.95, u'b': 0.9, u'type': u'TruncatedBetaDist', u'minVal': 0.1}, u'logmmreg': Struct{u'c': 1.0, u'beta': {u'intercept': 0.5063141361639452, u'sa.phenotypes.cov3': -0.4206565557494321, u'sa.phenotypes.cov2': 0.21050647010858073}, u'phi': 0.2}, u'popDist': [0.1, 0.2, 0.3, 0.4], u'nPops': 4, u'nVariants': 200, u'seed': 1, u'Fst': [0.02, 0.06, 0.04, 0.12], u'nSamples': 200}
